In [2]:
from requests import get
from bs4 import BeautifulSoup
import os

- Lets see if we can scrape all the top trending repos for the month for a language

In [58]:
url = 'https://github.com/trending/swift?since=monthly'

response = get(url)

In [59]:
soup = BeautifulSoup(response.content, 'html.parser')

In [60]:
print(soup.prettify())

<!DOCTYPE html>
<html lang="en">
 <head>
  <meta charset="utf-8"/>
  <link href="https://github.githubassets.com" rel="dns-prefetch"/>
  <link href="https://avatars0.githubusercontent.com" rel="dns-prefetch"/>
  <link href="https://avatars1.githubusercontent.com" rel="dns-prefetch"/>
  <link href="https://avatars2.githubusercontent.com" rel="dns-prefetch"/>
  <link href="https://avatars3.githubusercontent.com" rel="dns-prefetch"/>
  <link href="https://github-cloud.s3.amazonaws.com" rel="dns-prefetch"/>
  <link href="https://user-images.githubusercontent.com/" rel="dns-prefetch"/>
  <link crossorigin="anonymous" href="https://github.githubassets.com/assets/frameworks-53332a145deeeaaf8f21feef45138a08.css" integrity="sha512-UzMqFF3u6q+PIf7vRROKCCcSAIYk0CGPD1MvMAnv0X7Pqxc6MTt+l1mXE6StaVvPg+m/XgXexi1uO8P/zszHgA==" media="all" rel="stylesheet">
   <link crossorigin="anonymous" href="https://github.githubassets.com/assets/site-eb896b2962500cd7d5d2488a2f376084.css" integrity="sha512-64lrKWJQD

In [61]:
soup = BeautifulSoup(response.content, 'html.parser')

In [62]:
soup.title.text

'Trending Swift repositories on GitHub this month · GitHub'

In [63]:
article = soup.find('h1', class_ = 'h3 lh-condensed')

repo_name = article.text.replace('\n', '')
repo_name =repo_name.replace(' ', '')

In [64]:
repo_name

'rileytestut/AltStore'

In [65]:
h1s= soup.find_all('h1', class_ = 'h3 lh-condensed')

In [66]:
for elem in h1s:
    repo_name = elem.text
    repo_name = repo_name.replace(' ', '')
    repo_name = repo_name.replace('\n', '')
    print(repo_name)
    print()

rileytestut/AltStore

yichengchen/clashX

Alamofire/Alamofire

exelban/stats

dkhamsing/open-source-ios-apps

vapor/vapor

signalapp/Signal-iOS

danielgindi/Charts

shadowsocks/ShadowsocksX-NG

corona-warn-app/cwa-app-ios

apollographql/apollo-ios

apple/swift-package-manager

gao-sun/eul

cormiertyshawn895/Retroactive

Caldis/Mos

SwiftOldDriver/iOS-Weekly

dmytro-anokhin/url-image

pointfreeco/swift-snapshot-testing

slackhq/PanModal

yagiz/Bagel

ephread/Instructions

matteocrippa/awesome-swift

realm/SwiftLint

Juanpe/About-SwiftUI

appbrewery/Xylophone-iOS13



- The goal is to automate the process to get the top 25 repos for 4 different languages 

In [126]:
# option for period is 
# daily, weekly, monthly

# enhancing the function to take a list of language as input and return lists of url and
# a dict with language name

def get_top_repo(language_list, timeline):
    repo_links = []
    item = {}
    for element in language_list:
        url = f'https://github.com/trending/{element}?since={timeline}'
        response = get(url)
        soup = BeautifulSoup(response.content, 'html.parser')
        h1s = soup.find_all('h1', class_ = 'h3 lh-condensed')
        item.update({'language' : element})
        
        for elem  in h1s:
            repo_name = elem.get_text()
            repo_name = repo_name.replace('\n', '')
            repo_name = repo_name.replace(' ', '')
            links = 'https://github.com/' + repo_name
            repo_links.append(links)
            
        
    return repo_links, item
    

In [127]:
links, item = get_top_repo(['python','swift'], 'weekly')

In [128]:
item

{'language': 'swift'}

In [107]:
links

['https://github.com/corpnewt/gibMacOS',
 'https://github.com/Rudrabha/Wav2Lip',
 'https://github.com/microsoft/Bringing-Old-Photos-Back-to-Life',
 'https://github.com/facebookresearch/pifuhd',
 'https://github.com/ytdl-org/youtube-dl',
 'https://github.com/Asabeneh/30-Days-Of-Python',
 'https://github.com/trekhleb/learn-python',
 'https://github.com/public-apis/public-apis',
 'https://github.com/jerry-git/learn-python3',
 'https://github.com/munki/macadmin-scripts',
 'https://github.com/jackfrued/Python-100-Days',
 'https://github.com/kholia/OSX-KVM',
 'https://github.com/MrMimic/data-scientist-roadmap',
 'https://github.com/donnemartin/system-design-primer',
 'https://github.com/PyTorchLightning/pytorch-lightning',
 'https://github.com/danielgatis/rembg',
 'https://github.com/joke2k/faker',
 'https://github.com/mitre/caldera',
 'https://github.com/networkx/networkx',
 'https://github.com/aimhubio/aim',
 'https://github.com/willmcgugan/rich',
 'https://github.com/tychxn/jd-assistant',

In [81]:
for elem in links:

    url = 'https://github.com/yichengchen/clashX'
response = get(url)

soup = BeautifulSoup(response.content, 'html.parser')

In [82]:
print(soup)


<!DOCTYPE html>

<html lang="en">
<head>
<meta charset="utf-8"/>
<link href="https://github.githubassets.com" rel="dns-prefetch"/>
<link href="https://avatars0.githubusercontent.com" rel="dns-prefetch"/>
<link href="https://avatars1.githubusercontent.com" rel="dns-prefetch"/>
<link href="https://avatars2.githubusercontent.com" rel="dns-prefetch"/>
<link href="https://avatars3.githubusercontent.com" rel="dns-prefetch"/>
<link href="https://github-cloud.s3.amazonaws.com" rel="dns-prefetch"/>
<link href="https://user-images.githubusercontent.com/" rel="dns-prefetch"/>
<link crossorigin="anonymous" href="https://github.githubassets.com/assets/frameworks-53332a145deeeaaf8f21feef45138a08.css" integrity="sha512-UzMqFF3u6q+PIf7vRROKCCcSAIYk0CGPD1MvMAnv0X7Pqxc6MTt+l1mXE6StaVvPg+m/XgXexi1uO8P/zszHgA==" media="all" rel="stylesheet">
<link crossorigin="anonymous" href="https://github.githubassets.com/assets/site-eb896b2962500cd7d5d2488a2f376084.css" integrity="sha512-64lrKWJQDNfV0kiKLzdghJjhBJyD7

In [91]:
article = soup.find('article', itemprop = 'text')

In [93]:
article.get_text()

"\n\n\n  ClashX\n  \n\nA rule based proxy For Mac base on Clash.\nFeatures\n\nHTTP/HTTPS and SOCKS protocol\nSurge like configuration\nGeoIP rule support\nSupport Vmess/Shadowsocks/Socks5\nSupport for Netfilter TCP redirect\n\nInstall\nYou can download from Release page\nDownload ClashX Pro With enhanced mode and other advance feature at AppCenter for free permanently.\nDownload ClashX Pro Beta with Native Apple Silicon support at AppCenter-CXP-AppleSilicon\nBuild\n\n\nMake sure have python3 and golang installed in your computer.\n\n\nDownload deps\nbash install_dependency.sh\n\n\n\nBuild and run.\n\n\nConfig\nThe default configuration directory is $HOME/.config/clash\nThe default name of the configuration file is config.yaml. You can use your custom config name and switch config in menu Config section.\nCheckout Clash or SS-Rule-Snippet for Clash or lancellc's gitbook for more detail.\nAdvance Config\nChange the ports of ClashX\nPlease modify the config.yaml file generated by ClashX, 

In [ ]:
for elem in links:
    content = []
    url = elem
    response = get(url)
    soup = BeautifulSoup(response.content, 'html.parser')
    article = soup.find('article', itemprop = 'text')
    
    content.append(article.get_text())


    

In [ ]:
content